In [172]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

In [183]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
N = train.shape[0]

In [184]:
frac = 0.1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
#train = train.tail(n=int(N * frac))
train = train.sample(frac=frac, random_state=57)

N = train.shape[0]

train['week'] = train['week'] - train['shift']

for i in range(N):
    if train['week'].values[i] <= 0:
        train['week'].values[i] = (train['week'].values[i] + 52) % 53
        train['year'].values[i] = (train['year'].values[i] - 1)

train = train.drop(['shift'], axis=1)

train = train.sort(['year', 'week'])

X = train.drop(['Num', 'y'], axis=1)

y = train['y'].reshape(N, 1)
y = [elem[0] for elem in y]

/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/nimloth/anaconda3/envs/python27/lib/python2.7/site-packages/ipykernel/__main__.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [185]:
def smape(y, y_pred, **kwargs):
    y = np.array(y)
    y_pred = np.array(y_pred)
    ans = (np.abs(y - y_pred) * 1.0 / (np.abs(y) + np.abs(y_pred))).mean() * 100
    return ans * 2

from sklearn.metrics import make_scorer
scorer = make_scorer(score_func=smape)

In [186]:
tscv = TimeSeriesSplit(n_splits=3)

In [187]:
# X = X.drop(['item_id', 'year', 'week'], axis=1)
# X = np.array(X)
# X = [[a * b for a in x for b in x] + list(x ** 3) + list(x) for x in X]

# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# accuracy = cross_val_score(model, X, y, cv=tscv, scoring=scorer)
# print(accuracy)

In [188]:
for tress_number in [100]:
    for depth in [14]:
        for random_state in [43]:
            model = RandomForestRegressor(n_estimators=tress_number, max_depth=depth,
                                              random_state=random_state, criterion='mse')
            accuracy = cross_val_score(model, X, y, scoring=scorer, cv=tscv)
            print(accuracy, tress_number, depth, random_state)

(array([ 30.87590497,  26.36072864,  35.55006779]), 100, 14, 43)


In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

#model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
model.fit(X, y)

N = test.shape[0]

test['week'] = test['week'] - test['shift']

for i in range(N):
    if test['week'].values[i] <= 0:
        test['week'].values[i] = (test['week'].values[i] + 52) % 53
        test['year'].values[i] = (test['year'].values[i] - 1)

test = test.drop(['shift'], axis=1)

X = test.drop(['Num'], axis=1)

preds = model.predict(X)

print len(preds)
print len(sample_submission)

In [ ]:
sample_submission['y'] = preds

In [ ]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [ ]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)